# Database analysis - EDA

In this notebook we perform Exploratory Data Analysis on the sentence database.
Som of the files we are going to use are in the *input* folder (also in the drive), and the json files with the sentence-level information are in the S3 bucket.\


In [1]:
import boto3
import csv
import json

In [2]:
# Access to S3 bucket

# path = 'C:/Users/jordi/Documents/claus/'
# path = '/home/propietari/Documents/claus/'
path = 'C:/Users/hrpeg/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
language = "spanish"
model = "simple_transformers/"

In [4]:
# To execute just in case the process was cbroken and we need to continueu filling the json file

# path = 'C:/Users/hrpeg/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/output/'
# file_name = f'sentence_metadata_{model.split("/")[0]}{language}.json'
# file = path + file_name
# with open(file, 'r') as fp:
#     results_dict = json.load(fp)

In [9]:
in_prefix = f"{language}_documents/HSSC/{model}updated_sentences/"

start_point = 0
i = 0

results_dict = {'n_sentences': [], 'min_sent_length': [], 'max_sent_length': [], 'avg_sent_length': [],
                'number_of_incentives': [], 'Supplies': [], 'Loan': [], 'Fine': [], 'Direct payment': [], 
                'Technical assistance': [], 'Tax benefit': []}

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):#.:
    if ".json" in obj.key and "ing" not in obj.key:
        i += 1
        if i > start_point:
            print(i,  end='\r')
            name = obj.key.split("/")[-1].split("_")[0]
            flag = False
            sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
            for metadata, value in sentences[name]["metadata"].items():
                try:
                    results_dict[metadata].append(value)
                except:
                    pass

path = 'C:/Users/hrpeg/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/output/'
file_name = f'sentence_metadata_{model.split("/")[0]}_{language}.json'
file = path + file_name
with open(file, 'w') as fp:
    json.dump(results_dict, fp, indent = 4)

In [6]:
path = 'C:/Users/hrpeg/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/output/'
file_name = f'sentence_metadata_{model.split("/"[0])}_{language}.json'
file = path + file_name
with open(file, 'w') as fp:
    json.dump(results_dict, fp, indent = 4)

In [16]:
8857

{}

In [5]:
len(results_dict["n_sentences"])

7731

In [ ]:

#             incentive_counter = 0
#             sentences_counter = 0
#             min_sentence_length = 100000
#             max_sentence_length = 0
#             sum_sentence_length = 0
#             last_sentence_position = 0
#             incentives_dict = {'Supplies': 0,
#                                'Loan': 0,
#                                'Fine': 0,
#                                'Direct payment': 0,
#                                'Technical assistance': 0,
#                                'Tax benefit': 0}
#             for sentence_id in sentences[name]["sentences"]:
#                 sentences_counter += 1
#                 sentence = sentences[name]["sentences"][sentence_id]["text"]
#                 min_sentence_length, max_sentence_length, sum_sentence_length = sentence_stats(sentence, min_sentence_length, max_sentence_length, sum_sentence_length)
#                 incentive = sentences[name]["sentences"][sentence_id]["label"]  
#                 if len(incentive) > 1 and incentive[0] == "Incentive":
#                     incentive_counter += 1
#                     sentence_number = sentence_id.split("_")[2]
#                     policy = sentences[name]["sentences"][sentence_id]["label"][1]
#                     if last_sentence_position == 0:
#                         results_list.append([])
#                         last_sentence_position = sentence_number
#                         sentences[name]["sentences"][sentence_id]["closest_sentence"] = 0
#                         flag = True
#                     else:
#                         sentences[name]["sentences"][sentence_id]["closest_sentence"] = int(sentence_number) - int(last_sentence_position)
#                         last_sentence_position = sentence_number

#                     relative_position = int(sentence_number) / sentences[name]["metadata"]["n_sentences"]
#                     sentences[name]["sentences"][sentence_id]["relative_position"] = round(relative_position, 2)
#                     incentives_dict[policy] += 1
#     #                 print(incentive, "\n", sentence_id, " ** ", last_sentence_position)

#             # Updating the dictionary
#             sentences[name]["metadata"]["min_sent_length"] = min_sentence_length
#             sentences[name]["metadata"]["max_sent_length"] = max_sentence_length
#             sentences[name]["metadata"]["avg_sent_length"] = int(sum_sentence_length/sentences_counter)
#             sentences[name]["metadata"]["number_of_incentives"] = incentive_counter
#             for key, value in incentives_dict.items():
#                 sentences[name]["metadata"][key] = value
#             s3.Object(s3BucketName, f"{out_prefix}{name}_sents.json").put(Body = (json.dumps(sentences, indent = 4)))
#             # Creating the results list
#             if flag:
#                 documents_with_incentives += 1
#                 results_list[documents_with_incentives].append(name)
#                 results_list[documents_with_incentives].append(incentive_counter)
#                 for key, value in incentives_dict.items():
#                     results_list[documents_with_incentives].append(value)

# with open(f"../output/incentives_simple_transformers_{language}.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(results_list)